In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_excel('Online Retail.xlsx')  # importing data as a DataFrame

## Data Preprocessing


In [3]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
df.shape


(406829, 8)

In [9]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [10]:
df.dropna(inplace=True) # droping null values

In [11]:
df.shape

(406829, 8)

In [13]:
df.isnull().sum() # checking null values

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [22]:
df['Country'].value_counts()  # To select on which country we are going perform Colleborative filtring recommendation.

United Kingdom          361878
Germany                   9495
France                    8491
EIRE                      7485
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               1877
Portugal                  1480
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
USA                        291
Israel                     250
Unspecified                244
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58
Lebanon                     45
Lithuani

In [17]:
# I am taking only one country data as mention in pdf
new_df = df.copy()
new_df = df[df['Country'] == 'United Kingdom']
new_df.head()  # hear new_df is data of US country

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


### Recommendation System for US Country

In [18]:
df2 = new_df.copy()
df2 = df2[['CustomerID','InvoiceDate','StockCode','Description']]
df2.head()

,CustomerID,InvoiceDate,StockCode,Description
0,17850.0,2010-12-01 08:26:00,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,17850.0,2010-12-01 08:26:00,71053,WHITE METAL LANTERN
2,17850.0,2010-12-01 08:26:00,84406B,CREAM CUPID HEARTS COAT HANGER
3,17850.0,2010-12-01 08:26:00,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,17850.0,2010-12-01 08:26:00,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [60]:
# now we have Invoice Date in DateTime formet so we have to change it in Date formet
final_df= df2.copy()
final_df.InvoiceDate = final_df.InvoiceDate.dt.date.astype('str')
final_df.head()

,CustomerID,InvoiceDate,StockCode,Description
0,17850.0,2010-12-01,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,17850.0,2010-12-01,71053,WHITE METAL LANTERN
2,17850.0,2010-12-01,84406B,CREAM CUPID HEARTS COAT HANGER
3,17850.0,2010-12-01,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,17850.0,2010-12-01,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [61]:
# We will look for duplicate entries
final_df.duplicated().sum()

13214

In [62]:
final_df.drop_duplicates(inplace=True) # Droping Duplicates

In [63]:

final_df.duplicated().sum()

0

In [64]:
final_df.head()

,CustomerID,InvoiceDate,StockCode,Description
0,17850.0,2010-12-01,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,17850.0,2010-12-01,71053,WHITE METAL LANTERN
2,17850.0,2010-12-01,84406B,CREAM CUPID HEARTS COAT HANGER
3,17850.0,2010-12-01,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,17850.0,2010-12-01,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [65]:
final_df.iloc[0]['InvoiceDate']

'2010-12-01'

## Recommendation :
                                Recommend item to the given customer id for a given date.

In [114]:

def recommend_items(cust_id,date):
    
    customer_details = final_df[(final_df['CustomerID'] == cust_id) & (final_df['InvoiceDate'] == date)]
    if len(customer_details) == 0 :
        return f"There is no entry for customer {cust_id}"
    
    # extracting purchased items by customer on given date
    cust_items = customer_details['StockCode'].to_list()
    
    # extracting cust who purchased same item
    similiar_items_cust = final_df[final_df['StockCode'].isin(cust_items)]['CustomerID'].unique()
    
    # Recommending items on their occurance count
    recommended_items = final_df[(final_df['CustomerID'].isin(similiar_items_cust) 
                                  & 
                                  (~ final_df['StockCode'].isin(cust_items)))]['Description'].value_counts()
    
    top_5_recommendations = recommended_items[:5].index
    
    
    return top_5_recommendations

In [115]:
recommend_items(17850,'2010-12-01')

Index(['REGENCY CAKESTAND 3 TIER', 'JUMBO BAG RED RETROSPOT',
       'ASSORTED COLOUR BIRD ORNAMENT', 'PARTY BUNTING',
       'LUNCH BAG RED RETROSPOT'],
      dtype='object')

In [117]:
import pickle
pickle.dump(final_df,open('final_df.pkl','wb'))


5195

0

CustomerID
12346.0       1
12747.0     103
12748.0    4295
12749.0     231
12820.0      59
           ... 
18280.0      10
18281.0       7
18282.0      13
18283.0     721
18287.0      70
Name: StockCode, Length: 3950, dtype: int64

,CustomerID,StockCode,Quantity,UnitPrice,TotalPrice,date
61619,12346.0,23166,74215,1.04,77183.6,2011-01-18


(354839, 6)

,CustomerID,Quantity,UnitPrice,TotalPrice
CustomerID,1.000000,-0.002757,-0.000898,0.001251
Quantity,-0.002757,1.000000,-0.001363,0.889071
UnitPrice,-0.000898,-0.001363,1.000000,0.213243
TotalPrice,0.001251,0.889071,0.213243,1.000000
